Import required packages

In [4]:
import pandas as pd
import numpy as np 
from pyspark.sql import SparkSession
from langdetect import detect
from pyspark.sql.functions import when, lit, col, udf
from transformers import T5Tokenizer, T5ForConditionalGeneration
import datetime

In [5]:
# Helper functions
def isEnglish(text):
    try:
        if detect(text) != 'en':
            return "No"
        return "Yes"
    except:
        return "No"
    return "No"
# Convert function to UDF
isEnglishUDF = udf(lambda z: isEnglish(z))

In [6]:
# create a Spark session
spark = SparkSession.builder.master("local[1]").appName('Topic Summarization').getOrCreate()

In [7]:
# Read data
df = spark.read.format("csv").option("header","true").load("hdfs://namenode:9000/dis_materials/data_reddit.csv")

In [8]:
df.dtypes

[('created_utc', 'string'),
 ('ups', 'string'),
 ('subreddit_id', 'string'),
 ('link_id', 'string'),
 ('name', 'string'),
 ('score_hidden', 'string'),
 ('author_flair_css_class', 'string'),
 ('author_flair_text', 'string'),
 ('subreddit', 'string'),
 ('id', 'string'),
 ('removal_reason', 'string'),
 ('gilded', 'string'),
 ('downs', 'string'),
 ('archived', 'string'),
 ('author', 'string'),
 ('score', 'string'),
 ('retrieved_on', 'string'),
 ('body', 'string'),
 ('distinguished', 'string'),
 ('edited', 'string'),
 ('controversiality', 'string'),
 ('parent_id', 'string')]

In [8]:
# drop duplicates
df2 = df.dropDuplicates()

In [9]:
# drop null values
df2 = df2.na.drop(subset=["subreddit","subreddit_id","body"])

In [10]:
# Select vital columns
cols =["ups","subreddit_id","sub_reddit","score","body","controversiality","parent_id", "author_flair_css_class","author_flair_text","English"]

In [11]:
df_english = df2.withColumn("English",isEnglishUDF(col("body")))

In [12]:
df_english = df_english.drop("created_utc","link_id","name","score_hidden","id","removal_reason","gilded","retrieved_on","edited","controversiality")

In [13]:
df_english.withColumn('created_utc', f.date_format(df.epoch.cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

NameError: name 'f' is not defined

In [14]:
df_english = df_english.filter(df_english.English == "Yes")

In [ ]:
channels = df_english.select('subreddit').distinct().rdd.map(lambda r:r[0]).collect()

In [ ]:
pq=list(channels)

In [ ]:
import re 
def chanels():
    pq=list(channels)
    for i in pq:
        re.compile(r"[\w']+")
        if len(i)<3
        p=df_english[['subreddit'==i]]
    return p


In [14]:
df_english.show()

ERROR:root:KeyboardInterrupt while sending command.               (0 + 0) / 121]
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# We are using t5-base model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')